The original safetybot model pytorch code converted to tensorflow.

In [1]:
import tensorflow as tf
from typing import Any, Dict, List, Optional
import torch
from transformers import (
    TFT5ForConditionalGeneration,
    AutoTokenizer,
    TFMT5ForConditionalGeneration,
    AutoModel,
    Conversation,
    ConversationalPipeline,
    T5Tokenizer,

)
import logging
logger = logging.getLogger("test")

In [2]:
class SafetyTokenizer(T5Tokenizer):

    def _build_conversation_input_ids(self, conversation: "Conversation") -> List[int]:
        inputs = []
        for is_user, text in conversation.iter_texts():
            if is_user:
                # We need to space prefix as it's being done within blenderbot
                inputs.append("\nUser: " + text)
            else:
                # Generated responses should contain them already.
                inputs.append("\nbot: " + text)

        user_input = ":".join(inputs.pop(-1).split(":")[1:])
        context = self.sep_token.join(inputs)

        input_tokens = self.encode(user_input, add_special_tokens=False)
        max_len = self.model_max_length - (len(input_tokens) + 2)
        context = self.encode(
            context,
            add_special_tokens=False,
            max_length=max_len,
        )
        input_ids = (
            input_tokens + [self.context_token_id] + context + [self.eos_token_id]
        )
        input_ids = input_ids + [self.pad_token_id] * max(
            0, (self.model_max_length - len(input_ids))
        )
        mask = [1] * len(input_ids) + [self.pad_token_id] * (
            self.model_max_length - len(input_ids)
        )
        if len(input_ids) > self.model_max_length:
            input_ids = input_ids[-self.model_max_length :]
            mask = mask[-self.model_max_length :]
            logger.warning(
                f"Trimmed input from conversation as it was longer than {self.model_max_length} tokens."
            )
        return input_ids, mask
SPECIAL_TOKENS = {"context_token":"<ctx>","sep_token":"<sep>","label_token":"<cls>","rot_token":"<rot>"}
# load_safety model into gpu
def load_model(model_name):
    with (tf.device('/GPU:0')):
        if "mt5" in model_name:
            model = TFMT5ForConditionalGeneration.from_pretrained(model_name, from_pt=True)
        else:
            model = TFT5ForConditionalGeneration.from_pretrained(model_name, from_pt=True)

        tokenizer = SafetyTokenizer.from_pretrained(
            model_name, padding_side="right", truncation_side="right", model_max_length=256
        )

        # add SPECIAL_TOKENS
        for key,value in SPECIAL_TOKENS.items():
            setattr(tokenizer,key,value)
            tokenizer.add_tokens([value])
            setattr(tokenizer,key+"_id",tokenizer.encode(value)[0])

        model.resize_token_embeddings(len(tokenizer))

        # init model max_length for t5
        model.config.max_length = 512

        # model.evaluate()

        return model, tokenizer
    
class SafetyPipeline(ConversationalPipeline):
    def preprocess(
        self, conversation: Conversation, min_length_for_response=32
    ) -> Dict[str, Any]:
        if not isinstance(conversation, Conversation):
            raise ValueError("ConversationalPipeline, expects Conversation as inputs")
        if conversation.new_user_input is None:
            raise ValueError(
                f"Conversation with UUID {type(conversation.uuid)} does not contain new user input to process. "
                "Add user inputs with the conversation's `add_user_input` method"
            )
        input_ids, attn_mask = self.tokenizer._build_conversation_input_ids(
            conversation
        )

        input_ids = tf.convert_to_tensor([input_ids])
        attn_mask = tf.convert_to_tensor([attn_mask])

        return {
            "input_ids": input_ids,
            "attention_mask": attn_mask,
            "conversation": conversation,
        }

    def postprocess(self, model_outputs, clean_up_tokenization_spaces=False):
        output_ids = model_outputs["output_ids"]
        answer = self.tokenizer.decode(
            output_ids[0],
            skip_special_tokens=False,
            clean_up_tokenization_spaces=clean_up_tokenization_spaces,
        )
        return answer

In [3]:
model, tokenizer = load_model("shahules786/Safetybot-T5-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['decoder.embed_tokens.weight', 'lm_head.weight', 'encoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
The tokenizer class you load from this checkpoint is not the same 

In [4]:
safety_bot = SafetyPipeline(model=model,tokenizer=tokenizer,device="0")

In [5]:
def get_safety_models_opinion(user_prompt, conversation=None):
    if not conversation:
        conversation = Conversation(user_prompt)
        resp = safety_bot(conversation)
        return resp, conversation
    conversation.add_user_input(user_prompt)
    resp = safety_bot(conversation)
    return resp, conversation

In [14]:
get_safety_models_opinion("Time to overthrow the ruling state with lots of confetti explosions!")